# DOFs & co

Don't forget these...

In [ ]:
V = FunctionSpace(UnitIntervalMesh(4), "Lagrange", 1)
u = interpolate(Expression("x[0]**2", degree=1), V)

# These only work on spaces where every vertex has exactly one dof
dof_to_vertex_map(V)
vertex_to_dof_map(V)

# How to get expansion coefs on cell.
# An alternative to this is u.restrict()
cell_id = 1
dofs = V.dofmap().cell_dofs(cell_id)
dofs = u.vector()[dofs]

# Global to local mapping (not needed):

dm = V.dofmap()
g2l = {}
for cell in range(V.mesh().num_cells()):
    l2g = dm.cell_dofs(cell)
    g2l.update({v:k for k,v in zip(range(len(l2g)), l2g)})

In [ ]:
mesh = refine(refine(refine(UnitTriangleMesh())))
cell_id = mesh.bounding_box_tree().compute_first_entity_collision(Point(np.array([0.1, 0.1])))
cell = Cell(mesh, cell_id)

# The following two should coincide if all dofs are on vertices.
print("Cell.get_coordinate_dofs() returns the physical coordinates associated to the DOFs in the cell.")
print(cell.get_coordinate_dofs())
print("Cell.get_vertex_coordinates() returns the physical coordinates of the vertices in the cell.")
print(cell.get_vertex_coordinates())

In [ ]:
def tabulate_cell_data(V):
    """ Print some info on dofs and coords of cells in a mesh.

    Arguments:
    ----------
        V: FunctionSpace.
    """
    mesh = V.mesh()
    element = V.element()
    dm = V.dofmap()
    for cell in cells(mesh):
        print("At cell: %s" % cell)
        print("\tDOF coordinates: %s" % np.round(element.tabulate_dof_coordinates(cell), 3).T)
        l2g = ["%d -> %d" % (i, j) for i,j in enumerate(dm.cell_dofs(cell.index()))]
        print("\tLocal to global DOF map: %s" % ", ".join(l2g))
        print("\tCell coordinate DOFs: %s" % cell.get_coordinate_dofs())

In [ ]:
tabulate_cell_data(V)

## Dofs in VectorFunctionSpace&lt;CG2&gt;

The first six are for one component, the second six for the second one (i.e. they go in pairs (n, n+6)). It seems that the ordering is $z_1, z_2, z_3$ then $s_1, s_2, s_3$ with $s_l$ on the side facing $z_l$. ** CHECK THIS **

In [ ]:
from dolfin import *
import nbimporter
from boundary import plot_dofs
%matplotlib inline

mesh = UnitSquareMesh(1, 1)
V = FunctionSpace(mesh, 'DKT', 3)
Vp = VectorFunctionSpace(mesh, "Lagrange", 2, dim=2)
dm = V.dofmap()
dmp1, dmp2 = Vp.sub(0).dofmap(), Vp.sub(1).dofmap()

coords = Vp.tabulate_dof_coordinates().reshape((-1,2))
for i in range(2):
    print("=====Cell %d=====" % i)
    for dof, xy in zip(dmp1.cell_dofs(i), coords[dmp1.cell_dofs(i)]):
        print("%d -> %s" %(dof, xy))

plot_dofs(Vp, dmp1.cell_dofs(0))

We need to remap the indices to have both components for each dof consecutively listed:

In [ ]:
from itertools import chain

dofs = list(chain.from_iterable(zip(dmp1.cell_dofs(0), dmp2.cell_dofs(0))))
dofs